In [1]:
import numpy as np
import numpy.random as ra
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
N = 512
theta = ra.uniform(0,2*np.pi,size=(N,N))

def ZeroBC(i,j):
    if i >= 0 and i < N and j >= 0 and j < N:
        return theta[i,j]
    else:
        return 0

adjacency = [(0,1),(0,-1),(1,0),(-1,0)]

def Hloc(i,j,proptheta=None):
    if proptheta is None:
        proptheta = ZeroBC(i,j)
    return - sum([np.cos(proptheta - ZeroBC(i+a,j+b)) for (a,b) in adjacency])

def MetropolisUpdate(i,j,beta=1.1343):
    proptheta = ra.uniform(0,2*np.pi)
    if ra.uniform(0,1) < np.exp(- beta * (Hloc(i,j,proptheta) - Hloc(i,j))):
        theta[i,j] = proptheta

In [19]:
for frame in tqdm(range(3000)):
    to_update = ra.uniform(0,1,size=(N,N)) < 0.05
    for i in range(N):
        for j in range(N):
            if to_update[i,j]:
                MetropolisUpdate(i,j)
    np.save(f'data/{frame:06d}.npy',theta)

In [3]:
thetas = [np.load(f'data/{i:06d}.npy').reshape((N,N)) for i in range(3000)]

In [4]:
for frame in tqdm(range(3000)):
    plt.matshow(thetas[frame], vmin=0, vmax=2*np.pi, cmap='hsv')
    plt.axis('off')
    plt.savefig(f'images/{frame:06d}.png', bbox_inches='tight')
    plt.close('all')

100%|██████████| 3000/3000 [04:55<00:00, 10.15it/s]
